### Produção de Petroleo Mundial

Os dados analisados foram disponibilizados pela [Energy Information Administration - EIA - Official Energy Statistics from the U.S. Government](https://www.eia.gov/international/data/world/petroleum-and-other-liquids/annual-refined-petroleum-products-consumption?pd=5&p=0000000000000000000000000000000000g&u=0&f=A&v=mapbubble&a=-&i=none&vo=value&t=C&g=00000000000000000000000000000000000000000000000001&l=249-ruvvvvvfvtvnvv1vrvvvvfvvvvvvfvvvou20evvvvvvvvvvnvvvs0008&s=94694400000&e=1672531200000&ev=false&)

In [ ]:
import pandas as pd
import plotly.express as px
import re
from decimal import Decimal


In [ ]:
df = pd.read_csv('/content/producao_petroleo.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11730 entries, 0 to 11729
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Pais      11730 non-null  object
 1   Ano       11730 non-null  int64 
 2   Producao  11730 non-null  object
dtypes: int64(1), object(2)
memory usage: 275.0+ KB


In [ ]:
def clean_production_value(value):
    cleaned_value = str(value).replace(".", "").replace("--", "0")
    try:
        return int(cleaned_value)
    except ValueError:
        return None

df['Producao'] = df['Producao'].apply(clean_production_value)

In [ ]:
df['Producao_milhoes_bpd'] = df['Producao'] / Decimal('1000000')

In [ ]:
df.head()

,Pais,Ano,Producao,Producao_milhoes_bpd
0,Afghanistan,1973,0,0
1,Albania,1973,0,0
2,Algeria,1973,10969013698630100,10969013698.6301
3,American Samoa,1973,0,0
4,Angola,1973,162,0.000162


In [ ]:
#Anos de 2000 a 2023
anos_filtrados = [ano for ano in range(2000, 2024)]

df_filtrado = df[df['Ano'].isin(anos_filtrados)]

df_filtrado = df_filtrado.dropna(subset=['Producao_milhoes_bpd'])

df_filtrado['Producao_milhoes_bpd'] = df_filtrado['Producao_milhoes_bpd'].astype(float)

fig = px.choropleth(df_filtrado,
                   locations='Pais',
                   locationmode='country names',
                   color='Producao_milhoes_bpd',
                   hover_name='Pais',
                   color_continuous_scale='YlOrRd',
                   animation_frame='Ano',
                   range_color=[0, df_filtrado['Producao_milhoes_bpd'].max() * 1.2],
                   title='Produção de Petróleo (milhões de barris/ano) por País - 2000 a 2023',
                   labels={'Producao_milhoes_bpd': 'Produção (milhões de barris/ano)'},
                   projection='natural earth')

fig.update_layout(
    coloraxis_colorbar=dict(
        title="Produção",
        title_side="right"
    )
)

fig.show()

In [ ]:

df_filtrado = df[(df['Ano'] >= 2000) & (df['Ano'] <= 2023)]

df_grouped = df_filtrado.groupby('Pais')['Producao_milhoes_bpd'].mean().reset_index()

top_10 = df_grouped.sort_values('Producao_milhoes_bpd', ascending=False).head(10)

fig = px.bar(top_10, x='Pais', y='Producao_milhoes_bpd',
            color_discrete_sequence=["green"],
            labels={'Producao_milhoes_bbd': 'Produção média anual (milhões de barris/dia)'},
            title='Top 10 Maiores Produtores de Petróleo (2000 a 2023) | Produção Média Anual (milhões de barris/dia)')
fig.show()